Get Historical Volatility for your Stock

In [2]:
import yfinance as yf
import numpy as np

def get_historical_volatility():
    symbol = input("Enter stock symbol to estimate volatility: ").upper()
    days = int(input("Enter number of past days to calculate volatility: "))

    stock = yf.Ticker(symbol)
    data = stock.history(period=f"{days}d")

    if data.empty:
        print("No data found for this symbol. Please check the symbol and try again.")
        return None

    # Calculate daily log returns
    returns = np.log(data['Close'] / data['Close'].shift(1)).dropna()

    # Daily volatility
    daily_volatility = np.std(returns)

    # Annualize volatility for 252 trading days
    annual_volatility = daily_volatility * np.sqrt(252)

    print(f"\nEstimated Historical Volatility for {symbol}: {annual_volatility:.2f} ({annual_volatility*100:.2f}%)")
    return annual_volatility

# Example usage
sigma = get_historical_volatility()



Estimated Historical Volatility for AAPL: 0.66 (65.82%)


Calculate the risk-free rate using the US Treasury bond yield 

In [5]:
def get_risk_free_rate():
    """
    Asks the user for time to expiration (in years) and fetches an appropriate US Treasury yield.
    """
    try:
        # Ask the user how long until option expiration
        time_to_expiration_years = float(input("Enter the expected time to expiration (in years, e.g., 0.25 for 3 months): "))

        # Pick the right treasury bond based on expiration
        if time_to_expiration_years <= 0.5:
            ticker = "^IRX"  # 3-month T-Bill
        elif time_to_expiration_years <= 1:
            ticker = "^FVX"  # 5-year Note used as proxy (Yahoo doesn't have perfect 1-year ticker)
        elif time_to_expiration_years <= 2:
            ticker = "^UST2Y"  # 2-year Note
        elif time_to_expiration_years <= 5:
            ticker = "^FVX"  # 5-year Note
        elif time_to_expiration_years <= 10:
            ticker = "^TNX"  # 10-year Note
        else:
            ticker = "^TYX"  # 30-year Bond

        # Fetch the data
        bond = yf.Ticker(ticker)
        data = bond.history(period="1d")
        
        latest_rate = data['Close'].iloc[-1] / 100  # Convert percentage to decimal

        print(f"\nCurrent Risk-Free Rate based on your expiration ({time_to_expiration_years} years): {latest_rate:.4f} ({latest_rate*100:.2f}%)")
        return latest_rate, time_to_expiration_years

    except Exception as e:
        print("Failed to fetch risk-free rate. Please enter manually.")
        # Fallback manual input
        manual_rate = float(input("Enter risk-free rate manually (decimal, e.g., 0.045 for 4.5%): "))
        manual_time = float(input("Enter time to expiration manually (in years): "))
        return manual_rate, manual_time

r, T = get_risk_free_rate()



Current Risk-Free Rate based on your expiration (0.25 years): 0.0419 (4.19%)


Use Historical Volatility and US Treasury Yield (as risk-free rate) to calculate the theoretical price of the option using the Black-Scholes model.

In [3]:
import requests
import math
from scipy.stats import norm

#stock price from Yahoo Finance
def get_stock_price(symbol):
    import yfinance as yf
    stock = yf.Ticker(symbol)
    data = stock.history(period="1d")
    return data['Close'][0]

# Black-Scholes Model
def black_scholes_call_put(S, K, T, r, sigma):
    d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    call_price = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    put_price = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

    return call_price, put_price

# Plot Call Option Payoff
def plot_call_option_payoff(strike_price, option_price, stock_price_min, stock_price_max):
    stock_prices = np.linspace(stock_price_min, stock_price_max, 100)
    payoff = np.maximum(stock_prices - strike_price, 0) - option_price
    break_even = strike_price + option_price


# Main app
def main():
    print("Real-Time Option Pricing")
    symbol = input("Enter stock symbol (e.g., AAPL): ").upper()
    S = get_stock_price(symbol)
    print(f"Current Stock Price of {symbol}: ${S:.2f}")

    K = float(input("Enter strike price: "))
    T = float(input("Enter time to expiration (in years, e.g., 0.5 for 6 months): "))
    r = float(input("Enter risk-free rate (as decimal, e.g., 0.02 for 2%): "))
    sigma = float(input("Enter volatility (as decimal, e.g., 0.3 for 30%): "))

    call, put = black_scholes_call_put(S, K, T, r, sigma)

    print(f"Strike price: ${K:.2f}")
    print(f"Time to expiration: {T} years")
    print(f"Risk-free rate: {r * 100:.2f}%")
    print(f"Volatility: {sigma * 100:.2f}%")
    print(f"\nTheoretical Call Option Price: ${call:.2f}")
    print(f"Theoretical Put Option Price: ${put:.2f}")

    
if __name__ == "__main__":
    main()


Real-Time Option Pricing


C:\Users\matej\AppData\Local\Temp\ipykernel_26180\1145197250.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return data['Close'][0]


Current Stock Price of AAPL: $209.28
Strike price: $215.00
Time to expiration: 0.25 years
Risk-free rate: 4.50%
Volatility: 66.00%

Theoretical Call Option Price: $26.02
Theoretical Put Option Price: $29.33
